In [1]:
import sys
import os
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
#去除没有必要的警告
import warnings
warnings.filterwarnings(action="ignore",message="internal gelsd")
# 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif'] = [u'simHei']
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号, 注意['SimHei']对应这句不行
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
from PIL import Image
import numpy as np
import os

image_folder = './dataset/train'

# 获取文件夹中所有图片的路径
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png')]

# 加载所有图片
images = []
for path in image_paths:
    image = Image.open(path)
    image = image.convert('L')
    images.append(image)

# 将所有图片转换成Numpy数组
images_array = np.array([np.array(image) for image in images])


In [8]:
print(images_array.shape)

(1557, 512, 512)
